In [ ]:
# %pip install openpyxl
# %pip install pandas

In [2]:
from openpyxl import Workbook, load_workbook
from win32com import client
import shutil
import pandas as pd
import os
from pywintypes import com_error
import shutil
from pathlib import Path
from openpyxl import load_workbook


In [3]:
def write_block(df, eds, worksheet, row_start, col_start=6, row_slice=slice(None)):
    """Escribe un bloque de datos de base_poa en la hoja Excel."""
    df_filtered = df.loc[df.CodEs == eds].iloc[row_slice, 4:].values

    for row_offset, row in enumerate(df_filtered):
        for col_offset, valor in enumerate(row):
            worksheet.cell(row=row_start + row_offset, column=col_start + col_offset, value=valor)


def eds_file(base_poa, base_app_copec, jz, eds, dir_molde: Path, dir_destino: Path):
    """Genera archivo POA para concesionarios con datos de base_poa y base_app_copec."""

    # Copiar molde (más rápido que copy2 si no necesitas metadata exacta como permisos/fechas)
    shutil.copy(dir_molde, dir_destino)

    workbook = load_workbook(dir_destino)
    
    # Seleccionar hoja "Plan Teórico"
    if "Plan Teórico" in workbook.sheetnames:
        workbook.active = workbook.sheetnames.index("Plan Teórico")
        
    worksheet = workbook.active

    # Bloques de escritura predefinidos
    bloques = [
        (slice(0, 7), 17),
        (slice(7, 13), 25),
        (slice(13, 16), 32),
        (slice(16, None), 36),
    ]

    # Escritura en bloque (más eficiente que cell-by-cell dentro de write_block)
    for row_slice, row_start in bloques:
        write_block(base_poa, eds, worksheet, row_start, row_slice=row_slice)

    # Procesar base_app_copec solo si tiene datos
    df_app = base_app_copec.loc[base_app_copec.CodEs == eds]
    if not df_app.empty:
        row_start, col_start = 41, 6
        valores = df_app.iloc[0, 1:].replace({'-': 0}).round(3).to_numpy()
        for col_offset, valor in enumerate(valores, start=0):
            worksheet.cell(row=row_start, column=col_start + col_offset, value=valor)

    # Proteger hojas solo si es necesario (evita overhead si ya están protegidas)
    if not worksheet.protection.sheet:
        worksheet.protection.sheet = True
        
    if "Plan Definitivo" in workbook.sheetnames:
        plan_def = workbook["Plan Definitivo"]
        if not plan_def.protection.sheet:
            plan_def.protection.sheet = True

    # Guardar (usa overwrite para no duplicar handles en disco)
    workbook.save(dir_destino)
    

def eds_file_old(base_poa, base_app_copec, jz, eds, dir_molde: Path, dir_destino: Path):
    """Genera archivo POA para concesionarios con datos de base_poa y base_app_copec."""

    # Copiar molde
    shutil.copy2(dir_molde, dir_destino)

    workbook = load_workbook(dir_destino)
    worksheet = workbook.active

    # Escribir bloques definidos
    bloques = [
        (slice(0, 7), 17),   # filas 0–6 → fila Excel 17
        (slice(7, 13), 25),  # filas 7–12 → fila Excel 25
        (slice(13, 16), 32), # filas 13–15 → fila Excel 32
        (slice(16, None), 36) # desde fila 16 → fila Excel 36
    ]

    for row_slice, row_start in bloques:
        write_block(base_poa, eds, worksheet, row_start, row_slice=row_slice)

    # Cargar base_app_copec si tiene datos
    df_app = base_app_copec.loc[base_app_copec.CodEs == eds]
    if not df_app.empty:
        row_start, col_start = 41, 6
        for col_offset, valor in enumerate(df_app.iloc[0, 1:].replace({'-': 0}).values):
            worksheet.cell(row=row_start, column=col_start + col_offset, value=round(valor, 3))

    # Proteger hojas
    worksheet.protection.sheet = True
    workbook["Plan Definitivo"].protection.sheet = True

    # Guardar
    workbook.save(dir_destino)


def eds_file_final(base_poa, jz, eds, dir_molde: Path, dir_destino: Path):
    """Versión final de archivo POA (sin base_app_copec)."""

    shutil.copy2(dir_molde, dir_destino)

    workbook = load_workbook(dir_destino)
    worksheet = workbook.active

    bloques = [
        (slice(0, 7), 17),
        (slice(7, 13), 25),
        (slice(13, 16), 32),
        (slice(16, None), 36)
    ]

    for row_slice, row_start in bloques:
        write_block(base_poa, eds, worksheet, row_start, row_slice=row_slice)

    if "Plan Definitivo" in workbook.sheetnames:
        plan_def = workbook["Plan Definitivo"]
        if not plan_def.protection.sheet:
            plan_def.protection.sheet = True
            
    workbook.save(dir_destino)


# ---- Diccionario global para sum_prop ----
DICC_PROP = {
    ('DSL', 'Cupón Electrónico'): 0.00004724,
    ('DSL', 'FF.AA'): 0.00000137,
    ('DSL', 'Cuenta Empresa'): 0.00001175,
    ('DSL', 'TAE'): 0.0000678,
    ('DSL', 'TCT'): 0.00014023,
    ('DSL', 'Venta Propia - Camión de Reparto'): 0.00000634,
    ('DSL', 'Venta Propia - Isla'): 0.00024819,
    ('GAS', 'Cupón Electrónico'): 0.00000837,
    ('GAS', 'FF.AA'): 0.0000031,
    ('GAS', 'G93 - VP'): 0.00029,
    ('GAS', 'G95 - VP'): 0.0000944,
    ('GAS', 'G97 - VP'): 0.00006226,
    ('GAS', 'Cuenta Empresa'): 0.00000345,
    ('GASES', 'GLP'): 0.00000105,
    ('GASES', 'GNC'): 0.00000115,
    ('KER', 'Cuenta Empresa'): 0.00000002,
    ('KER', 'Venta Propia - Camión de Reparto'): 0.0000004,
    ('KER', 'Venta Propia - Isla'): 0.00001291
}


def sum_prop(prod, form_com):
    """Devuelve el valor de proporción según producto y forma de comercialización."""
    return DICC_PROP.get((prod, form_com), 0)


## Archivos Iniciales en base al año en curso

In [9]:
# ---- Directorios base ----
base_dir = Path(r"G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA 2026\POA Comercial - Carga Inicial")

dir_base = base_dir / "Carga Inicial" / "Archivos por Zona Respaldo"
dir_maestro_eds = base_dir / "Carga Inicial" / "Maestro EDS.xlsx"
# dir_archivo_base = base_dir / "Carga Inicial" / "(Paso-1) ArchivosSinProyMacro" / "base_poa_vol_inicial - python.xlsx"
# dir_archivo_app_copec = base_dir / "Carga Inicial" / "(Paso-1) ArchivosSinProyMacro" / "base_poa_app_copec.xlsx"
# dir_archivo_base = base_dir / "Carga Inicial" / "(Paso-3) Mercado" / "base_poa_vol_inicial - python.xlsx"
# dir_archivo_base = base_dir / "Carga Inicial" / "(Paso-3) Mercado" /  "base_poa_vol_inicial - python - Carga Inicial_SinAfectaciones.xlsx"
dir_archivo_base = Path(r"G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA 2026") / "VolumenFinalValores - python.xlsx"
dir_archivo_app_copec = base_dir / "Carga Inicial" / "(Paso-3) Mercado" / "base_poa_app_copec.xlsx"

# ---- Configuración ----
zonas = [209, 210, 211, 212, 213, 214, 215, 216, 217, 
218, 219, 220, 221, 230, 231, 232, 233, 234, 
235, 240, 241, 242, 243, 244, 245, 246, 247, 
248, 249]  # , 211, ..., 248

# zonas = [214]

# ---- Carga maestro ----
maestro_eds = pd.read_excel(dir_maestro_eds, usecols=[0, 5], sheet_name="Consolidado VF")
estacionales = pd.read_excel(dir_maestro_eds, usecols=[0, 12], names=["CodEs", "JZ"], sheet_name="Estacionales")

# Concatenar ambos
maestro_eds = pd.concat([maestro_eds, estacionales], ignore_index=True)
maestro_eds["CodEs"] = maestro_eds["CodEs"].fillna(0).astype(int)
maestro_eds["JZ"] = maestro_eds["JZ"].fillna(0).astype(int)

# ---- Bases de datos ----
# base_poa = pd.read_excel(dir_archivo_base, skiprows=1, sheet_name="Carga Inicial")
base_poa = pd.read_excel(dir_archivo_base, skiprows=1, sheet_name="VolumenFinal")
print(base_poa.columns.tolist())
base_app_copec = pd.read_excel(dir_archivo_app_copec)
# base_poa["CodEs"] = base_poa["CodEs"].fillna(0).astype(int)
base_poa["CodEs"] = base_poa["Estación "].fillna(0).astype(int)
base_poa.drop(columns=["Estación "], inplace=True)

print(base_poa.head())

# ---- Detalles forma comercial ----
details_form_com = [
    ("DSL", "Cupón Electrónico"),
    ("DSL", "FF.AA"),
    ("DSL", "Cuenta Empresa"),
    ("DSL", "TAE"),
    ("DSL", "TCT"),
    ("DSL", "Venta Propia - Camión de Reparto"),
    ("DSL", "Venta Propia - Isla"),
    ("GAS", "Cupón Electrónico"),
    ("GAS", "FF.AA"),
    ("GAS", "G93 - VP"),
    ("GAS", "G95 - VP"),
    ("GAS", "G97 - VP"),
    ("GAS", "Cuenta Empresa"),
    ("GASES", "GLP"),
    ("GASES", "GNC"),
    ("KER", "Cuenta Empresa"),
    ("KER", "Venta Propia - Camión de Reparto"),
    ("KER", "Venta Propia - Isla"),
]

# ---- Columnas proyectadas ----
years = [2024, 2025, 2026]
months = [f"{m:02}" for m in range(1, 13)]
cols_proy = [f"{y}-{m:02}" for y in years for m in months]

# ---- Asegurar que las columnas proyectadas sean numéricas ----
base_poa[cols_proy] = base_poa[cols_proy].apply(pd.to_numeric, errors='coerce')

# ---- Aplicar sum_prop por cada combinación ----
for prod, form_com in details_form_com:
    mask = (base_poa["Producto"] == prod) & (base_poa["Forma Comercialización"] == form_com)
    incremento = sum_prop(prod, form_com)
    print(f"Producto: {prod}, Forma Comercialización: {form_com}, Incremento: {incremento:.8f}")
    base_poa.loc[mask, cols_proy] = base_poa.loc[mask, cols_proy].add(incremento)

base_poa


['Estación ', 'Producto', 'Forma Comercialización', 'Estado', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09', '2025-10', '2025-11', '2025-12', '2026-01', '2026-02', '2026-03', '2026-04', '2026-05', '2026-06', '2026-07', '2026-08', '2026-09', '2026-10', '2026-11', '2026-12']
  Producto Forma Comercialización  Estado  2024-01  2024-02  2024-03  2024-04  \
0      DSL         Cuenta Empresa  Normal   17.093   17.840   15.675   16.847   
1      DSL      Cupón Electrónico  Normal   65.066   57.079   61.543   66.675   
2      DSL                  FF.AA  Normal    1.388    0.471    1.341    1.425   
3      DSL                    TAE  Normal    0.000    0.000    0.000    0.000   
4      DSL                    TCT  Normal    0.000    0.000    0.000    0.000   

   2024-05  2024-06  2024-07  ...    2026-04    2026-05

,Producto,Forma Comercialización,Estado,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,...,2026-04,2026-05,2026-06,2026-07,2026-08,2026-09,2026-10,2026-11,2026-12,CodEs
0,DSL,Cuenta Empresa,Normal,1.709301e+01,1.784001e+01,1.567501e+01,1.684701e+01,20.975012,1.821801e+01,23.215012,...,3.069650e+01,2.879885e+01,3.129597e+01,2.524406e+01,3.907461e+01,3.198991e+01,3.275574e+01,3.359623e+01,3.349689e+01,10004
1,DSL,Cupón Electrónico,Normal,6.506605e+01,5.707905e+01,6.154305e+01,6.667505e+01,62.909047,5.868905e+01,65.320047,...,6.334447e+01,6.160682e+01,6.292587e+01,5.922958e+01,5.808427e+01,5.758117e+01,6.043905e+01,6.282416e+01,5.911095e+01,10004
2,DSL,FF.AA,Normal,1.388001e+00,4.710014e-01,1.341001e+00,1.425001e+00,0.781001,1.032001e+00,0.316001,...,6.641399e-01,8.664406e-01,1.310515e+00,1.196393e+00,8.280460e-01,6.345409e-01,9.671610e-01,1.123312e+00,9.359308e-01,10004
3,DSL,TAE,Normal,6.780000e-05,6.780000e-05,6.780000e-05,6.780000e-05,0.000068,6.780000e-05,0.000068,...,6.780000e-05,6.780000e-05,6.780000e-05,6.780000e-05,6.780000e-05,6.780000e-05,6.780000e-05,6.780000e-05,6.780000e-05,10004
4,DSL,TCT,Normal,1.402300e-04,1.402300e-04,1.402300e-04,1.402300e-04,0.000140,1.402300e-04,0.000140,...,1.402300e-04,1.402300e-04,1.402300e-04,1.402300e-04,1.402300e-04,1.402300e-04,1.402300e-04,1.402300e-04,1.402300e-04,10004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12937,GASES,GLP,Normal,1.413300e+01,1.064900e+01,1.528100e+01,1.970900e+01,21.825001,2.332900e+01,25.526001,...,1.050000e-06,1.050000e-06,1.050000e-06,1.050000e-06,1.050000e-06,1.050000e-06,1.050000e-06,1.050000e-06,1.050000e-06,60999
12938,GASES,GNC,Normal,1.150000e-06,1.150000e-06,1.150000e-06,1.150000e-06,0.000001,1.150000e-06,0.000001,...,1.150000e-06,1.150000e-06,1.150000e-06,1.150000e-06,1.150000e-06,1.150000e-06,1.150000e-06,1.150000e-06,1.150000e-06,60999
12939,KER,Cuenta Empresa,Normal,2.000000e-08,2.000000e-08,2.000000e-08,2.000000e-08,0.010000,2.000000e-08,0.018000,...,2.000000e-08,2.000000e-08,2.000000e-08,2.000000e-08,2.000000e-08,2.000000e-08,2.000000e-08,2.000000e-08,2.000000e-08,60999
12940,KER,Venta Propia - Camión de reparto,Normal,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,60999


In [10]:
base_poa.groupby(by=['Producto'])['2025-01'].sum()

Producto
DSL      318606.569421
GAS      264571.388876
GASES       941.939582
KER         225.776297
Name: 2025-01, dtype: float64

In [12]:
# Generación archivos
output_dir = base_dir / "Carga Inicial" / "Archivos por Zona Respaldo"
dir_molde = base_dir / "Carga Inicial" / "(Paso-1) ArchivosSinProyMacro" / "Formato POA JZ - Concesionarios.xlsx"


# --- Crear carpetas por zona ---output_dir = base_dir / "Carga Inicial" / "Archivos por Zona Respaldo"
for zona in zonas:
    directory = Path(dir_base) / str(zona)
    directory.mkdir(parents=True, exist_ok=True)

eds_falla = []

print(maestro_eds)

# --- Iterar maestro_eds agrupado por JZ ---
for cont, (jz, df_jz) in enumerate(maestro_eds.groupby("JZ"), start=1):
    for eds in df_jz["CodEs"]:
        try:
            
            # if(eds not in [40531,40532,40533,40534,60292,60294,60295,60296,60297,60298]):
            #     continue
            
            print(eds)
            
            output_file = output_dir / str(jz) / f"{eds} - Combustibles.xlsx"
            print(output_file)
            
            # if output_file.exists():
            #     print(f"⚠️ Ya existe {output_file}, se omite.")
            #     continue
            
            eds_file(base_poa, base_app_copec, jz, eds, dir_molde, dir_destino=output_file)
            print(f"{cont}: EDS {eds} en JZ {jz}")
        except Exception as e:
            eds_falla.append((eds, jz, str(e)))
            print(f"❌ Fallo EDS {eds} en JZ {jz} -> {e}")

     CodEs   JZ
0    10004  220
1    10006  220
2    10009  220
3    10021  221
4    10024  220
..     ...  ...
727  60017  214
728  60685  213
729  20155  246
730  20194  247
731  20198  247

[732 rows x 2 columns]
60076
G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA 2026\POA Comercial - Carga Inicial\Carga Inicial\Archivos por Zona Respaldo\209\60076 - Combustibles.xlsx
1: EDS 60076 en JZ 209
60207
G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA 2026\POA Comercial - Carga Inicial\Carga Inicial\Archivos por Zona Respaldo\209\60207 - Combustibles.xlsx
1: EDS 60207 en JZ 209
60230
G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA 2026\POA Comercial - Carga Inicial\Carga Inicial\Archivos por Zona Respaldo\209\60230 - Combustibles.xlsx
1: EDS 60230 en JZ 209
60259
G:\Unidades compartidas\2_Planificación Comercial - EDS\4. POA\POA Gestión\POA 2026\POA Comercial - Carga Inicial\Carga Inicial\Archivos

In [ ]:
eds_falla

In [ ]:
eds_file(base_poa, base_app_copec, 214, 60293)

## Archivos Finales en base a Carga Inicial

In [ ]:
dir_maestro_eds = r"G:\Unidades compartidas\2_Planificación Comercial - Combustibles\4. POA\POA Gestión\POA 2024\Carga Inicial\Maestro EDS.xlsx"

maestro_eds = pd.read_excel(dir_maestro_eds, usecols=[0, 5], sheet_name='Consolidado')
estacionales = pd.read_excel(dir_maestro_eds, usecols=[0, 12], names=['CodEs', 'JZ'], sheet_name='Estacionales')
maestro_eds = maestro_eds.append(estacionales)


dir_archivo_base_final = r"G:\Unidades compartidas\2_Planificación Comercial - Combustibles\4. POA\POA Gestión\POA 2024\Carga Inicial\Carga Inicial V4 EDS.xlsx"

base_poa_final = pd.read_excel(dir_archivo_base_final, skiprows=1, sheet_name="Carga Inicial - Volumen", usecols="A:AN")

In [ ]:
details_form_com = [['DSL', 'Cupón Electrónico'], ['DSL', 'FF.AA'], ['DSL', 'Muevo Empresa'], ['DSL', 'TAE'], ['DSL', 'TCT'], ['DSL', 'Venta Propia - Camión de Reparto'],
['DSL', 'Venta Propia - Isla'], ['GAS', 'Cupón Electrónico'], ['GAS', 'FF.AA'], ['GAS', 'G93 - VP'], ['GAS', 'G95 - VP'], ['GAS', 'G97 - VP'], ['GAS', 'Muevo Empresa'],
['GASES', 'GLP'], ['GASES', 'GNC'], ['KER', 'Muevo Empresa'], ['KER', 'Venta Propia - Camión de Reparto'], ['KER', 'Venta Propia - Isla']]

cols_proy = ['2022-1',	'2022-2',	'2022-3',	'2022-4',	'2022-5',	'2022-6',	'2022-7',	'2022-8',	'2022-9',	'2022-10',	'2022-11',	'2022-12',	'2023-1',	'2023-2',	'2023-3',	'2023-4',	'2023-5',	'2023-6',	'2023-7',	'2023-8',	'2023-9',	'2023-10',	'2023-11',	'2023-12',	'2024-1',	'2024-2',	'2024-3',	'2024-4',	'2024-5',	'2024-6',	'2024-7',	'2024-8',	'2024-9',	'2024-10',	'2024-11',	'2024-12']
for option in details_form_com:
   base_poa_final.loc[(base_poa_final.Producto == option[0]) & (base_poa_final['Forma Comercialización'] == option[1]), (cols_proy)] = base_poa_final.loc[(base_poa_final.Producto == option[0]) & (base_poa_final['Forma Comercialización'] == option[1]), (cols_proy)].add(sum_prop(option[0], option[1]))


base_poa_final

In [ ]:
#eds_file_final(base_poa_final, 213, 60444)

In [ ]:
cont = 1
for jz in maestro_eds.JZ.unique():
    for eds in maestro_eds[maestro_eds.JZ == jz].CodEs:
        eds_file_final(base_poa_final, jz, eds)
        print(cont, eds, jz)
        cont += 1